Swagger codegen is so broken it's amazing

```

# java 10 doesn't work, use java8 because reflection issues (??)
sudo apt install openjdk-8-jdk-headless
export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/
export PATH=${JAVA_HOME}/bin:$PATH # not sure if needed

# build it
git clone https://github.com/swagger-api/swagger-codegen
cd swagger-codegen
mvn clean package

# generate client
cd ../eve_market_analysis
java -jar ../swagger-codegen/modules/swagger-codegen-cli/target/swagger-codegen-cli.jar \
    generate -i https://esi.evetech.net/latest/swagger.json -l python --library asyncio -o python-client

# fix yet another bug
find python-client/ -name \*.py | xargs sed -i 's/\<async:/async_:/g'
find python-client/ -name \*.py | xargs sed -i 's/\<async=/async_=/g'

# conda install
source activate eve
cd python-client
python3 setup.py install
```

In [1]:
%%capture
# only run this once
"""
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
"""

In [2]:
import swagger_client as eve
import asyncio

In [3]:
class ESIClient:
    def __init__(self, cls):
        self.cls = cls
    
    async def __aenter__(self):
        self.client = self.cls(eve.ApiClient())
        # not sure if this actually works
        self.client.api_client.user_agent = "nickgarvey Eve Dev"
        return self.client

    async def __aexit__(self, exc_type, exc, tb):
        # generated code don't close the pool_manager, so do it ourselves
        await self.client.api_client.rest_client.pool_manager.close()


# not used
async def gen_region(universe_client, region_id):
    return await universe_client.get_universe_regions_region_id(region_id)


async def gen_all_regions():
    async with ESIClient(eve.UniverseApi) as universe_client:
        region_ids = await universe_client.get_universe_regions()
        regions = await universe_client.post_universe_names(region_ids)
    return {r['id']: r['name'] for r in regions}


loop = asyncio.new_event_loop()
region_names = loop.run_until_complete(gen_all_regions())

In [4]:
async def gen_prices():
    async with ESIClient(eve.MarketApi) as market_client:
        prices = await market_client.get_markets_prices()
        return {p['type_id']: p for p in prices}

loop = asyncio.new_event_loop()
prices = loop.run_until_complete(gen_prices())

In [5]:
# ORDER_PAGE_SIZE = 1000  # seems to be the page size

# CHUNK_SIZE = 100
# chunked_ids = [item_ids[i:i + CHUNK_SIZE] for i in range(0, len(item_ids), CHUNK_SIZE)]


import random

item_ids = random.sample(list(prices.keys()), 3)


async def gen_region_history(market_client, region_id, item_id):
    try:
        price_history = await market_client.get_markets_region_id_history(region_id, item_id)
        return (item_id, price_history)
    except eve.rest.ApiException as e:
        if e.status != 404:
            raise
        return (item_id, [])


async def gen_all_history_region(market_client, region_id):
    history = []
    # TODO many requests in parallel
    for chunk in [[i] for i in item_ids]:
        histories = await asyncio.gather(*[
            gen_region_history(market_client, region_id, item_id)
            for item_id in chunk
        ])
        history.extend(histories)
    return history
    

async def gen_all_orders(regions):
    async with ESIClient(eve.MarketApi) as market_client:
        item_groups = await market_client.get_markets_groups()
        region_ids = list(regions.keys())
        market_data = await asyncio.gather(*[
            gen_all_history_region(market_client, region_id)
            for region_id in region_ids
        ])
        return {
            region_id: dict(region_data)
            for region_id, region_data
            in zip(region_ids, market_data)
        }


loop = asyncio.new_event_loop()
regions = dict(list(region_names.items())[0:1])
region_prices = loop.run_until_complete(gen_all_orders(regions))

import json
print(json.dumps(region_prices, indent=4)[:1000] + '...')

{
    "10000001": {
        "47636": [],
        "1136": [
            {
                "average": 900000.0,
                "date": "2017-12-11",
                "highest": 900000.0,
                "lowest": 900000.0,
                "order_count": 1,
                "volume": 1
            },
            {
                "average": 900000.01,
                "date": "2018-02-09",
                "highest": 900000.01,
                "lowest": 900000.01,
                "order_count": 1,
                "volume": 1
            },
            {
                "average": 900000.0,
                "date": "2018-04-20",
                "highest": 900000.0,
                "lowest": 900000.0,
                "order_count": 1,
                "volume": 1
            },
            {
                "average": 900000.0,
                "date": "2018-05-19",
                "highest": 900000.0,
                "lowest": 900000.0,
                "order_count": 1,
                "volume":